In [33]:
# Mengimpor modul drive dari pustaka google.colab untuk mengakses Google Drive
from google.colab import drive

# Mount Google Drive ke dalam folder /content/drive
drive.mount('/content/drive')

# Mengimpor fungsi train_test_split dari modul sklearn.model_selection
from sklearn.model_selection import train_test_split

# Mengimpor XGBRegressor dari modul xgboost
from xgboost import XGBRegressor


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
# Mengimpor pustaka pandas dan memberikan alias 'pd' untuk kemudahan penggunaan
import pandas as pd

In [35]:
# Membaca file CSV ('melb_data.csv') dan menyimpannya ke dalam DataFrame 'DT'
DT = pd.read_csv('/content/master.csv')


In [36]:
# Menampilkan konten DataFrame 'DT'
DT


,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_per_year,gdp_per_capita,generation,Unnamed: 12
0,Albania,1987,male,15-24 years,21.0,312900,6.710000,Albania1987,NaN,"2,15,66,24,900",796.000000,Generation X,7720.0
1,Albania,1987,male,35-54 years,16.0,308000,5.190000,Albania1987,NaN,"2,15,66,24,900",796.000000,Silent,7020.0
2,Albania,1987,female,15-24 years,14.0,289700,4.830000,Albania1987,NaN,"2,15,66,24,900",796.000000,Generation X,NaN
3,Albania,1987,male,75+ years,1.0,21800,4.590000,Albania1987,NaN,"2,15,66,24,900",796.000000,G.I. Generation,4056.0
4,Albania,1987,male,25-34 years,9.0,274300,3.280000,Albania1987,NaN,"2,15,66,24,900",796.000000,Boomers,5646.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31751,Ukraine,2017,female,75+ years,256.0,44831135,0.571032,Ukraine2017,0.785458,1.12E+11,2638.326055,G.I. Generation,NaN
31752,United Kingdom,2017,female,75+ years,104.0,66058859,0.157435,United Kingdom2017,0.914735,2.70E+12,40857.755580,G.I. Generation,NaN
31753,United States of America,2017,female,75+ years,501.0,325122128,0.154096,United States of America2017,0.918620,1.95E+13,60109.655730,G.I. Generation,NaN
31754,Uruguay,2017,female,75+ years,14.0,3422200,0.409094,Uruguay2017,0.816745,64233966861,18690.893840,G.I. Generation,NaN


In [37]:
# Menampilkan informasi ringkas tentang struktur dan tipe data dalam DataFrame 'DT'
DT.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31756 entries, 0 to 31755
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   country            31756 non-null  object 
 1   year               31756 non-null  int64  
 2   sex                31756 non-null  object 
 3   age                31756 non-null  object 
 4   suicides_no        30556 non-null  float64
 5   population         31756 non-null  int64  
 6   suicides/100k pop  31756 non-null  float64
 7   country-year       31756 non-null  object 
 8   HDI for year       12300 non-null  float64
 9   gdp_per_year       31756 non-null  object 
 10  gdp_per_capita     31756 non-null  float64
 11  generation         31756 non-null  object 
 12  Unnamed: 12        6 non-null      float64
dtypes: float64(5), int64(2), object(6)
memory usage: 3.1+ MB


In [38]:
# Menghitung jumlah nilai NaN (missing values) pada kolom 'Bedroom2', 'Bathroom', dan 'Car'
nan_count_Bedroom2 = DT['suicides_no'].isna().sum()
nan_count_Bathroom = DT['HDI for year'].isna().sum()
nan_count_Car = DT['population'].isna().sum()

# Menampilkan jumlah nilai NaN untuk setiap kolom
print("Jumlah NaN pada suicides_no:", nan_count_Bedroom2)
print("Jumlah NaN pada HDI for year:", nan_count_Bathroom)
print("Jumlah NaN pada population:", nan_count_Car)

Jumlah NaN pada suicides_no: 1200
Jumlah NaN pada HDI for year: 19456
Jumlah NaN pada population: 0


In [39]:
# Menghapus baris yang memiliki nilai NaN pada kolom 'Car' dari DataFrame 'DT'
DT.dropna(subset=['suicides_no'], inplace=True)


In [40]:
# Menghitung kembali jumlah nilai NaN pada kolom 'Car' setelah penghapusan
carnt = DT['suicides_no'].isna().sum()

# Menampilkan jumlah nilai NaN untuk kolom 'Car' setelah penghapusan
print("Jumlah NaN pada suicides_no setelah penghapusan:", carnt)


Jumlah NaN pada suicides_no setelah penghapusan: 0


In [41]:
# Menampilkan ukuran DataFrame 'DT' setelah penghapusan nilai NaN pada kolom 'Car'
DT.shape


(30556, 13)

In [42]:
# Mengubah tipe data kolom 'Bedroom2', 'Bathroom', dan 'Car' menjadi tipe data integer
DT['suicides_no'] = DT['suicides_no'].astype(int)


In [43]:
# Memilih kolom-kolom tertentu ('Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt') dan menyimpannya dalam DataFrame 'X'
cols_to_use = ['gdp_per_capita', 'population', 'suicides_no']
X = DT[cols_to_use]

In [44]:
#membagi dataset menjadi subset pelatihan dan subset validasi
y=DT.suicides_no
X_train,X_valid,y_train,y_valid=train_test_split(X,y,random_state=42,test_size=0.3)

In [45]:
# Memisahkan dataset menjadi subset pelatihan dan pengujian menggunakan train_test_split
# 'X' adalah fitur (kolom-kolom tertentu) dan 'y' adalah target (kolom 'Price')

# Import fungsi train_test_split
from sklearn.model_selection import train_test_split

# Memisahkan dataset menjadi subset pelatihan dan pengujian
# 'random_state' digunakan untuk memastikan reproduktibilitas hasil
# 'test_size' menentukan ukuran subset pengujian (0.3 berarti 30% dari data akan menjadi data pengujian)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=42, test_size=0.3)


In [46]:
# Membuat model XGBoost Regressor
my_model = XGBRegressor()

# Melatih model menggunakan subset pelatihan
my_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [47]:
# Mengimpor modul mean_absolute_error dari sklearn.metrics
from sklearn.metrics import mean_absolute_error

# Melakukan prediksi menggunakan model yang telah dilatih pada subset pengujian
prediction = my_model.predict(X_valid)

# Menghitung dan mencetak Mean Absolute Error (MAE) antara prediksi dan nilai sebenarnya
print("Mean Absolute Error: " + str(mean_absolute_error(prediction, y_valid)))


Mean Absolute Error: 8.063294573704846


In [48]:
# Membuat model XGBoost Regressor dengan n_estimators=500
my_model_0 = XGBRegressor(n_estimators=500)

# Melatih model menggunakan subset pelatihan
my_model_0.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [49]:
# Melakukan prediksi menggunakan model yang memiliki n_estimators=500 pada subset pengujian
prediction_0 = my_model_0.predict(X_valid)

# Menghitung dan mencetak Mean Absolute Error (MAE) antara prediksi dan nilai sebenarnya
print("Mean Absolute Error (n_estimators=500): " + str(mean_absolute_error(prediction_0, y_valid)))


Mean Absolute Error (n_estimators=500): 8.063364198553566


In [50]:
# Membuat model XGBoost Regressor dengan n_estimators=100
my_model_1 = XGBRegressor(n_estimators=100)

# Melatih model menggunakan subset pelatihan
my_model_1.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [51]:
# Melakukan prediksi menggunakan model yang memiliki n_estimators=100 pada subset pengujian
prediction_1 = my_model_1.predict(X_valid)

# Menghitung dan mencetak Mean Absolute Error (MAE) antara prediksi dan nilai sebenarnya
print("Mean Absolute Error (n_estimators=100): " + str(mean_absolute_error(prediction_1, y_valid)))


Mean Absolute Error (n_estimators=100): 8.063294573704846


In [52]:
# Melakukan pelatihan model XGBoost Regressor dengan berbagai nilai n_estimators
from xgboost import XGBRegressor

# Daftar nilai n_estimators yang akan diuji
n_estimators_list = [100, 150, 200, 250, 300]

# Menyimpan hasil Mean Absolute Error (MAE) untuk setiap nilai n_estimators
mae_results = []

# Melakukan iterasi untuk setiap nilai n_estimators
for n_estimators in n_estimators_list:
    # Membuat model XGBoost Regressor dengan n_estimators saat ini
    model_grid = XGBRegressor(n_estimators=n_estimators)

    # Melatih model menggunakan subset pelatihan
    model_grid.fit(X_train, y_train)

    # Melakukan prediksi pada subset pengujian
    predictions = model_grid.predict(X_valid)

    # Menghitung MAE untuk model dengan n_estimators saat ini
    mae = mean_absolute_error(predictions, y_valid)

    # Menampilkan hasil MAE
    print(f"n_estimators: {n_estimators}, Mean Absolute Error: {mae}")

    # Menyimpan hasil MAE dalam daftar
    mae_results.append(mae)


n_estimators: 100, Mean Absolute Error: 8.063294573704846
n_estimators: 150, Mean Absolute Error: 7.975423528897258
n_estimators: 200, Mean Absolute Error: 8.050812324468374
n_estimators: 250, Mean Absolute Error: 8.038145777764079
n_estimators: 300, Mean Absolute Error: 8.050614338214604


In [53]:
# Membuat model XGBoost Regressor dengan n_estimators=100 dan menggunakan early stopping
# Parameter early_stopping_rounds=5 menentukan berapa banyak iterasi tanpa peningkatan yang akan menyebabkan penghentian dini
# eval_set=[(X_valid, y_valid)] menyediakan subset validasi untuk pemantauan early stopping
# verbose=False untuk mengurangi tampilan keluaran
my_model_2=XGBRegressor(n_estimators=100)
# Melatih model dengan early stopping
my_model_2.fit(X_train,y_train,early_stopping_rounds=5,eval_set=[(X_valid,y_valid)],verbose=False)
# Melakukan prediksi pada subset pengujian
predictions_2=my_model_2.predict(X_valid)
# Menghitung dan mencetak Mean Absolute Error (MAE) antara prediksi dan nilai sebenarnya
print("Mean Abasolute Error: " +str(mean_absolute_error(predictions_2,y_valid)))

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Mean Abasolute Error: 7.999947484577684


In [54]:
# Membuat model XGBoost Regressor dengan n_estimators=100, learning_rate=0.52, dan n_jobs=4
my_model_3=XGBRegressor(n_estimators=100,learning_rate=0.52,n_jobs=4)
# Melatih model
my_model_3.fit(X_train,y_train)
# Melakukan prediksi pada subset pengujian
predictions_3=my_model_3.predict(X_valid)
# Menghitung dan mencetak Mean Absolute Error (MAE) antara prediksi dan nilai sebenarnya
print("Mean Abasolute Error: " +str(mean_absolute_error(predictions_3,y_valid)))

Mean Abasolute Error: 8.239919724474422
